In [1]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
loader = TextLoader("./demo2.txt")
docs = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=40,
    separators=["\n", "。", "！", "？", "，", "、", ""]
)

In [4]:
texts = text_splitter.split_documents(docs)

In [5]:
embeddings_model = OpenAIEmbeddings()

In [6]:
db = FAISS.from_documents(texts, embeddings_model)

In [7]:
retriever = db.as_retriever()

In [8]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [9]:
memory = ConversationBufferMemory(return_messages=True,
                                  memory_key='chat_history',
                                  output_key='answer')
qa = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory,
    chain_type="map_reduce"
)
qa.invoke({"chat_history": memory, "question": "卢浮宫这个名字怎么来的？"})

{'chat_history': [HumanMessage(content='卢浮宫这个名字怎么来的？'),
  AIMessage(content='根据法国百科全书辞典大拉鲁斯百科全书的说法，卢浮宫这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国： lupara）。')],
 'question': '卢浮宫这个名字怎么来的？',
 'answer': '根据法国百科全书辞典大拉鲁斯百科全书的说法，卢浮宫这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国： lupara）。'}

In [10]:
memory = ConversationBufferMemory(return_messages=True,
                                  memory_key='chat_history',
                                  output_key='answer')
qa = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory,
    chain_type="refine"
)
qa.invoke({"chat_history": memory, "question": "卢浮宫这个名字怎么来的？"})

{'chat_history': [HumanMessage(content='卢浮宫这个名字怎么来的？'),
  AIMessage(content='根据提供的新背景信息，卢浮宫这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国： lupara）。卢浮宫是一座位于法国巴黎市中心的塞纳河边的博物馆，原是建于12世纪末至13世纪初的王宫，现在是一所世界上最大的艺术博物馆之一，也是参观人数最多的博物馆之一。1793年8月10日，正值君主制灭亡一周年，卢浮宫正式命名为“中央艺术博物馆”开放。在大革命期间，卢浮宫展示了来自皇室、教堂、贵族和地方政府等处没收来的艺术品。后来，为了扩大收藏，共和国每年投入约100,000里弗维持博物馆的经营，并开始从欧洲各地和梵蒂冈等地带来作品。在拿破仑时代，卢浮宫继续展示各种珍贵的艺术品，成为了法国艺术和文化的重要中心。')],
 'question': '卢浮宫这个名字怎么来的？',
 'answer': '根据提供的新背景信息，卢浮宫这个名字来源于与狼狩猎巢穴的联系（拉丁语：lupus，下帝国： lupara）。卢浮宫是一座位于法国巴黎市中心的塞纳河边的博物馆，原是建于12世纪末至13世纪初的王宫，现在是一所世界上最大的艺术博物馆之一，也是参观人数最多的博物馆之一。1793年8月10日，正值君主制灭亡一周年，卢浮宫正式命名为“中央艺术博物馆”开放。在大革命期间，卢浮宫展示了来自皇室、教堂、贵族和地方政府等处没收来的艺术品。后来，为了扩大收藏，共和国每年投入约100,000里弗维持博物馆的经营，并开始从欧洲各地和梵蒂冈等地带来作品。在拿破仑时代，卢浮宫继续展示各种珍贵的艺术品，成为了法国艺术和文化的重要中心。'}

In [11]:
memory = ConversationBufferMemory(return_messages=True,
                                  memory_key='chat_history',
                                  output_key='answer')
qa = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory,
    chain_type="map_rerank"
)
qa.invoke({"chat_history": memory, "question": "卢浮宫这个名字怎么来的？"})

/usr/local/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'chat_history': [HumanMessage(content='卢浮宫这个名字怎么来的？'),
  AIMessage(content='这个名字来源于与狼狩猎巢穴的联系')],
 'question': '卢浮宫这个名字怎么来的？',
 'answer': '这个名字来源于与狼狩猎巢穴的联系'}